# Purpose
1. Hold ListWise Ranking Algorthim
2. Train Algorthim
3. Release trained Algorthim as a file

In [1]:
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Phase 1:
#### Get train and test data

# Phase 2
##### Define Model

In [2]:
class RankingModel(tfrs.Model):

    def __init__(self, loss):
        super().__init__()
        embedding_dimension = 32

        # User embeddings
        self.user_embeddings = tf.keras.Sequential([ # only user ID is used not User features
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids),  # need unique_user_ids
            tf.keras.layers.Embedding(
                len(unique_user_ids) + 2, embedding_dimension)
        ])

        # Restaurand embeddings
        self.restaurant_embeddings = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_restaurants),  # need unique_restaurants
            tf.keras.layers.Embedding(
                len(unique_restaurants) + 2, embedding_dimension)
        ])

        # Compute predictions
        self.score_model = tf.keras.Sequential([
            # Learn multiple dense layers.
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

        self.task = tfrs.tasks.Ranking(
            loss=loss,
            metrics=[
                tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
                tf.keras.metrics.RootMeanSquaredError()
            ]
        )

    def call(self, features):  # features Type: Dict{}, Key: UserID, BookID, Value: Tensor
        user_embeddings = self.user_embeddings(features["UserID"])
        restaurant_embeddings = self.restaurant_embeddings(features["BookID"])

        # The embeddings are concatenated
        list_length = features["BookID"].shape[1]
        user_embedding_repeated = tf.repeat(
            tf.expand_dims(user_embeddings, 1), [list_length], axis=1)
        concatenated_embeddings = tf.concat(
            [user_embedding_repeated, restaurant_embeddings], 2)

        return self.score_model(concatenated_embeddings)

    def compute_loss(self, features, training=False):
        labels = features.pop("Rating")
        scores = self(features)
        return self.task(
            labels=labels,
            predictions=tf.squeeze(scores, axis=-1),
        )

# Phase 3
#### Train Model

In [3]:
# cached_train = train.shuffle(buffer_size).batch(8192).cache() # define buffer_size

NameError: name 'train' is not defined

In [ ]:
# listwise_model = RankingModel(tfr.keras.losses.ListMLELoss())
# listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
# listwise_model.fit(cached_train, epochs=5, verbose=True)